In [5]:
import matplotlib.pyplot as plt
import matplotlib.animation
%matplotlib ipympl
import numpy as np
import pandas as pd

from scipy.spatial import KDTree, cKDTree
from scipy.ndimage import uniform_filter1d
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit

import time
import joblib
from tqdm import tqdm
import trackpy as tp
from numba import njit, prange

from yupi import Trajectory
import yupi.graphics as yg
import yupi.stats as ys

from utility import get_imsd, get_imsd_windowed, get_emsd, get_emsd_windowed, fit_hist, MB_2D,\
                    normal_distr, get_trajs, speed_windowed, theta_windowed, get_smooth_trajs, get_velocities
                   
show_verb = True
save_verb = True
anim_show_verb = False

ModuleNotFoundError: No module named 'utility'

In [7]:
rawTrajs = pd.read_parquet("../../tracking/results/pre_merge_tracking.parquet")
red_particle_idx = 17
rawTrajs.loc[rawTrajs.particle != red_particle_idx, ["color"]] = "#00007F"
rawTrajs.loc[rawTrajs.particle == red_particle_idx, ["color"]] = "#FF0000"
colors = rawTrajs.loc[rawTrajs.frame == 0, 'color'].values
nDrops = len(rawTrajs.loc[rawTrajs.frame==0])
nFrames = max(rawTrajs.frame) + 1
print(f"nDrops:{nDrops}")
print(f"nFrames:{nFrames} --> {nFrames/10:.2f} s")


# WINDOWED ANALYSIS PARAMETERS
window = 3200 # 320 s
stride = 100 # 10 s
print(f"window of {window/10} s, stride of {stride/10} s")
startFrames = np.arange(0, nFrames-window, stride, dtype=int)
endFrames = startFrames + window
nSteps = len(startFrames)
print(f"number of steps: {nSteps}")


# step 10 with a 10 fps video --> 1 s
units = "px/s"
default_kwargs_blue = {"color": "#00FFFF", "ec": (0, 0, 0, 0.6), "density": True}
default_kwargs_red = {"color": "#EE4B2B", "ec": (0, 0, 0, 0.6), "density": True}

nDrops:50
nFrames:32000 --> 3200.00 s
window of 320.0 s, stride of 10.0 s
number of steps: 288


In [4]:
class ParticleMetrics:
    def __init__(self, trajectory_df):
        self.trajectory = trajectory_df
        
    def compute_imsd(self, maxLagtime):
        pxDimension = 1 # has to be fixed 
        fps = 10 # fps of the video
        lag = np.arange(1, maxLagtime + 1, 1)/fps
        imsd, fit, pw_exp = get_imsd(rawTrajs, pxDimension, fps, maxLagtime, nDrops)
        return np.array(msd)
    
    def compute_velocity_distribution(self, num_bins=20):
        velocities = self.trajectory.groupby('particle').diff()
        speed = np.sqrt(np.sum(np.square(velocities), axis=1))
        hist, bins = np.histogram(speed, bins=num_bins, density=True)
        hist = hist.reshape(-1, self.trajectory['particle'].nunique())
        return hist, bins
    
    def plot_msd(self, time_lags):
        msd = self.compute_msd(time_lags)
        mean_msd = np.mean(msd, axis=1)
        std_msd = np.std(msd, axis=1)
        plt.errorbar(time_lags, mean_msd, yerr=std_msd, fmt='-o')
        plt.xlabel('Time lag')
        plt.ylabel('Mean squared displacement')
        plt.show()
        
    def plot_velocity_distribution(self, num_bins=20):
        hist, bins = self.compute_velocity_distribution(num_bins)
        mean_hist = np.mean(hist, axis=1)
        std_hist = np.std(hist, axis=1)
        plt.errorbar(bins[:-1], mean_hist, yerr=std_hist, fmt='o')
        plt.xlabel('Speed')
        plt.ylabel('Probability density')
        plt.show()


In [8]:
metrics = ParticleMetrics(rawTrajs)